In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# Definir la arquitectura de la red neuronal
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Definir transformaciones de datos para preprocesamiento
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Cargar el conjunto de datos
dataset = ImageFolder('archive', transform=transform)

# Dividir el conjunto de datos en entrenamiento, validación y prueba
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

# Crear cargadores de datos
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Definir hiperparámetros
input_size = 28 * 28 * 3 # Tamaño de la imagen de entrada (224x224 píxeles con 3 canales)
hidden_size = 512  # Tamaño de la capa oculta
num_classes = len(dataset.classes)  # Número de clases
print ("Tamaño de entrada____: ", input_size)
print ("Tamaño capa oculta___: ", hidden_size)
print ("Número de clases_____: ", num_classes)

# Crear una instancia del modelo MLP
model = MLP(input_size, hidden_size, num_classes)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Función para entrenar el modelo
def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    return total_loss, accuracy

# Función para evaluar el modelo en el conjunto de validación
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100.0 * correct / total
        return total_loss, accuracy

# Función para evaluar el modelo en el conjunto de prueba
def test(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100.0 * correct / total
        return total_loss, accuracy

# Entrenamiento y evaluación del modelo
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy = validate(model, val_loader, criterion)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# Evaluación final del modelo en el conjunto de prueba
test_loss, test_accuracy = test(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Tamaño de entrada____:  2352
Tamaño capa oculta___:  512
Número de clases_____:  1
Epoch [1/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [2/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [3/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [4/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [5/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [6/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [7/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [8/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [9/10], Train Loss: 0.0000, Train Accuracy: 100.00%, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch [10/10], Train Loss: